In [1]:
import os
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import plotly.graph_objects as go

import shutil
import random

from scipy import signal
from collections import OrderedDict

In [2]:
path = r'c:\Users\dhers\OneDrive - UBA\NOR videos\example\WIN_20240408_15_33_58_Pro_LDLC_Resnet50_STORM_2Oct14shuffle2_snapshot_900_raw.h5'

df = pd.read_hdf(path)
scorer = df.columns.levels[0][0]
bodyparts = df.columns.levels[1].to_list()

df_raw = df.copy()
df_raw

scorer    DLC_Resnet50_STORM_2Oct14shuffle2_snapshot_900              \
bodyparts                                          obj_1               
coords                                                 x           y   
0                                             239.260101  116.001480   
1                                             235.716339  108.047020   
2                                             231.359924  105.329994   
3                                             226.839081  104.823784   
4                                             222.185516  101.538452   
...                                                  ...         ...   
26995                                         199.660309  517.616943   
26996                                         199.239563  517.270142   
26997                                         177.282166  586.185913   
26998                                         179.651459  581.249084   
26999                                         180.031540  574.117371   

scorer                                                               \
bodyparts                  obj_2                               nose   
coords    likelihood           x           y likelihood           x   
0           0.098863  239.180130  113.169128   0.036814  193.945358   
1           0.124321  239.977570  111.171707   0.043672  184.027069   
2           0.122331  238.028442  109.023560   0.030166  177.280609   
3           0.174671  232.877686  107.882484   0.025065  169.590134   
4           0.160937  241.108978  103.775620   0.020834  165.139954   
...              ...         ...         ...        ...         ...   
26995       0.038793  181.029709  583.148682   0.066653  188.263321   
26996       0.040236  117.793625  719.516357   0.026604  186.818924   
26997       0.039650  192.720566  619.292664   0.041794  182.836990   
26998       0.039736  191.885056  621.896851   0.037312  178.541672   
26999       0.040418  194.202942  624.306763   0.027699  173.845261   

scorer                                        ...                         \
bodyparts                              L_ear  ...       body      tail_1   
coords              y likelihood           x  ... likelihood           x   
0           56.798836   0.954959  205.841461  ...   0.571567  242.936127   
1           54.334347   0.990080  196.096863  ...   0.608531  245.445251   
2           51.736359   1.000000  188.854797  ...   0.652895  250.675385   
3           48.681740   0.999521  181.548401  ...   0.571487  253.894608   
4           43.344978   0.856249  174.372849  ...   0.602432  254.094406   
...               ...        ...         ...  ...        ...         ...   
26995      493.172485   0.902663  162.214325  ...   0.768954  526.717896   
26996      493.432281   0.881270  163.218475  ...   0.799251  190.349701   
26997      485.503448   0.894484  162.442261  ...   0.798432  187.402161   
26998      478.138062   0.889085  161.562561  ...   0.715413  366.368652   
26999      471.294617   0.885591  159.562164  ...   0.769018  366.335846   

scorer                                                               \
bodyparts                             tail_2                          
coords              y likelihood           x           y likelihood   
0          161.754456   0.887418  224.371033  214.656586   0.645795   
1          158.629517   0.870531  231.188217  210.551376   0.648817   
2          153.441849   0.896456  236.578934  206.116257   0.711690   
3          147.874313   0.916277  243.421158  201.290848   0.646825   
4          140.568863   0.986176  249.778687  194.741043   0.664393   
...               ...        ...         ...         ...        ...   
26995      700.387024   0.447562  306.996582  645.235718   0.646892   
26996      636.935547   0.617620  301.565369  641.982605   0.631920   
26997      632.597229   0.503000  301.216888  640.956421   0.591748   
26998      674.270447   0.572909  300.885895  635.785217   0.608768   
26999   

In [3]:
# Try different filtering parameters
med_window = 3
sigma, n_sigmas = 0.6, 2
N = int(2 * n_sigmas * sigma + 1)

# Gaussian kernel
gauss_kernel = signal.windows.gaussian(N, sigma)
gauss_kernel = gauss_kernel / sum(gauss_kernel)

pad_width = (len(gauss_kernel) - 1) // 2
gauss_kernel

array([0.16637851, 0.66724298, 0.16637851])

In [4]:
# Iterate over each body part
for point in bodyparts:

    mean = df[scorer, point, 'likelihood'].mean()
    std_dev = df[scorer, point, 'likelihood'].std()
    limit = mean - 1*std_dev

    print(f'{point}\t{limit}')
    
    # Create mask where likelihood is below the threshold
    mask = df[scorer, point, 'likelihood'] < limit
    # Set 'x' and 'y' to NaN where likelihood is below the threshold
    df.loc[mask, (scorer, point, ['x', 'y'])] = np.nan

    # Iterate over x and y axes
    for axis in ['x', 'y']:
        # Access the specific axis column for this body part
        # column = df[scorer, point, axis]

        # Check if there are any non-NaN values left to interpolate
        if df[scorer, point, axis].notna().sum() > 1:
            # Interpolate using the pchip method, limit_area='inside' ensures interpolation only within valid data
            df[scorer, point, axis] = df[scorer, point, axis].interpolate(method='pchip', limit_area='inside')
            
            # Forward fill the remaining NaN values
            df[scorer, point, axis] = df[scorer, point, axis].ffill()
            
            # Apply median filter
            df[scorer, point, axis] = signal.medfilt(df[scorer, point, axis], kernel_size=med_window)
            
            # Pad the median filtered data to mitigate edge effects
            padded = np.pad(df[scorer, point, axis], pad_width, mode='edge')
            # Apply convolution
            smooth = signal.convolve(padded, gauss_kernel, mode='valid')
            # Trim the padded edges to restore original length
            df[scorer, point, axis] = smooth[:len(df[scorer, point, axis])]
            
            if 'obj' in point:
                df[scorer, point, axis] = 0.000000
            
df

L_ear	0.7000066339969635
R_ear	0.7015392109751701
body	0.5219476073980331
head	0.7107638716697693
neck	0.7463705167174339
nose	0.609702467918396
obj_1	-0.015930280089378357
obj_2	0.008107203990221024
tail_1	0.7998179197311401
tail_2	0.514712505042553
tail_3	0.7562787681818008


scorer    DLC_Resnet50_STORM_2Oct14shuffle2_snapshot_900                  \
bodyparts                                          obj_1                   
coords                                                 x    y likelihood   
0                                                    0.0  0.0   0.098863   
1                                                    0.0  0.0   0.124321   
2                                                    0.0  0.0   0.122331   
3                                                    0.0  0.0   0.174671   
4                                                    0.0  0.0   0.160937   
...                                                  ...  ...        ...   
26995                                                0.0  0.0   0.038793   
26996                                                0.0  0.0   0.040236   
26997                                                0.0  0.0   0.039650   
26998                                                0.0  0.0   0.039736   
26999                                                0.0  0.0   0.040418   

scorer                                                              \
bodyparts obj_2                        nose                          
coords        x    y likelihood           x           y likelihood   
0           0.0  0.0   0.036814  184.027069   54.334347   0.954959   
1           0.0  0.0   0.043672  182.904603   53.902097   0.990080   
2           0.0  0.0   0.030166  177.123545   51.660385   1.000000   
3           0.0  0.0   0.025065  170.129249   48.302040   0.999521   
4           0.0  0.0   0.020834  165.451411   43.902712   0.856249   
...         ...  ...        ...         ...         ...        ...   
26995       0.0  0.0   0.066653  188.068940  493.500320   0.902663   
26996       0.0  0.0   0.026604  186.396732  491.939747   0.881270   
26997       0.0  0.0   0.041794  182.784850  485.553969   0.894484   
26998       0.0  0.0   0.037312  178.474939  478.224901   0.889085   
26999       0.0  0.0   0.027699  174.626643  472.433219   0.885591   

scorer                 ...                                                \
bodyparts       L_ear  ...       body      tail_1                          
coords              x  ... likelihood           x           y likelihood   
0          196.096863  ...   0.571567  243.353591  158.629517   0.887418   
1          194.891939  ...   0.608531  245.897969  157.766400   0.870531   
2          188.844094  ...   0.652895  250.340812  153.378647   0.896456   
3          181.570171  ...   0.571487  253.358998  147.585162   0.916277   
4          174.788419  ...   0.602432  253.473335  141.057744   0.986176   
...               ...  ...        ...         ...         ...        ...   
26995      163.083364  ...   0.768954  675.795166  253.397720   0.447562   
26996      162.570216  ...   0.799251  675.795166  253.397720   0.617620   
26997      162.295898  ...   0.798432  675.795166  253.397720   0.503000   
26998      161.376101  ...   0.715413  675.795166  253.397720   0.572909   
26999      159.894987  ...   0.769018  675.795166  253.397720   0.653326   

scorer                                                                \
bodyparts      tail_2                             tail_3               
coords              x           y likelihood           x           y   
0          225.505266  210.551376   0.645795  215.574638  264.298798   
1          230.950884  209.813468   0.648817  220.850255  263.794991   
2          236.820433  206.051321   0.711690  226.424495  260.485402   
3          243.340515  201.003945   0.646825  232.999226  253.960319   
4          249.300841  195.068781   0.664393  238.071825  248.529047   
...               ...         ...        ...         ...         ...   
26995      306.694737  644.932705   0.646892  292.528106  738.528131   
26996      302.411026  642.353118   0.631920  289.685627  739.312253   
26997      301.274868  640.266779   0.591748  287.304850  740.292210   
26998      301.161818  63

In [5]:
import plotly.graph_objects as go

fig = go.Figure()

fig.add_trace(go.Scatter(x=df.index, y=df[scorer]['nose']['x']))
fig.add_trace(go.Scatter(x=df_raw.index, y=df_raw[scorer]['nose']['x']))

In [6]:
# Save the updated DataFrame to an HDF5 file
out_path = r'c:\Users\dhers\OneDrive - UBA\NOR videos\example\WIN_20240408_15_33_58_Pro_LDLC_Resnet50_STORM_2Oct14shuffle2_snapshot_900.h5'
df.to_hdf(out_path, key='df', mode='w')